In [1]:
# cspell: ignore kernelspec ipykernel codemirror ipython nbconvert pygments nbformat #

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import json

In [ ]:
from autoware_smart_mpc_trajectory_follower.scripts import drive_functions
import python_simulator

In [ ]:
# header of auto_test_result_*.csv
header = [
    "max_abs_lateral_error", 
    "straight_line_abs_max_lateral_error",
    "max_abs_vel_error", 
]


plot_param_list = [
    # [label, header_index, unit_for_plot, nominal_value]
    ["steer_bias", 0, "[deg]", 0.0] ,
    #["steer_rate_lim", 0, "[rad/s]", python_simulator.steer_rate_lim],
    #["vel_rate_lim", 2, "[m/ss]", python_simulator.vel_rate_lim],
    ["steer_dead_band", 0, "[rad]", python_simulator.steer_dead_band],
    ["wheel_base",0, "[m]", drive_functions.L],
    # ["adaptive_gear_ratio_coef", 2,"[-]", None],
    ["acc_time_delay",2, "[s]", drive_functions.acc_time_delay],
    ["steer_time_delay",0, "[s]", drive_functions.steer_time_delay],
    ["acc_time_constant",2, "[s]", drive_functions.acc_time_constant],
    ["steer_time_constant",0, "[s]", drive_functions.steer_time_constant],
    ["accel_map_scale",2, "[-]", None],
    ["acc_scaling",2, "[-]", 1],
    ["steer_scaling",0, "[-]", 1],
    ["vehicle_type", 0, "[-]",0],
]
print(plot_param_list)
print("len(plot_param_list)",len(plot_param_list))


In [ ]:
mem_diff = True
root_dir = "auto_test"

def get_dir_name(param_name, index, intermediate):
    dir_name = root_dir + "/test_pp_aided_sim_"
    if not intermediate:
        dir_name += "trained_"
    
    dir_name += param_name + "_" + str(index) + "th"
    if mem_diff:
        dir_name += "_mem_diff"
    return dir_name
fig = plt.figure(figsize=(30, 18), tight_layout=True)

ax_list = []
for i in range(len(plot_param_list)):
    ax_list.append(plt.subplot(4, 4, i+1))


df_info_list = [ 
    { 
    "intermediate" : False, 
    "label_name" : "poly(selected)+NN (final model)"},
    { 
    "intermediate" : True, 
    "label_name" : "poly(selected)+NN (intermediate model)"},
]

plot_intermediate = []

cmap = plt.get_cmap("tab10") 
for l in range(len(df_info_list)):
    #df = pd.read_csv(df_info_list[l]["file_name"], header=None,sep=",")
    for i in range(len(plot_param_list)):
        index = 0
        x = []
        y = []
        while os.path.exists(get_dir_name(plot_param_list[i][0],index,df_info_list[l]["intermediate"])):
            with open(get_dir_name(plot_param_list[i][0],index,df_info_list[l]["intermediate"])+"/auto_test_performance_result.json","r") as f:
                data=json.load(f)
            x.append(list(list(data.values())[7].values())[0])
            y.append(list(data.values())[plot_param_list[i][1]])
            index += 1
        if len(x) > 0:
            x = np.array(x)
            y = np.array(y)
            if plot_param_list[i][0]=="steer_bias":
                x *= 180.0/np.pi
        if i==0:
            ax_list[i].plot(x,y,"o-",label=df_info_list[l]["label_name"],color=cmap(l))
        else:
            ax_list[i].plot(x,y,"o-",label=df_info_list[l]["label_name"],color=cmap(l))

for i in range(len(plot_param_list)):
    ax_list[i].set_xlabel(plot_param_list[i][0] + " " + plot_param_list[i][2],fontsize=20)
    ax_list[i].set_ylabel(header[plot_param_list[i][1]],fontsize=20)
    ax_list[i].grid()

    if plot_param_list[i][3] is not None:
        ax_list[i].plot([plot_param_list[i][3],plot_param_list[i][3]],[-0.1,3],"k--")

    # lateral_error
    if 2==plot_param_list[i][1]:
        ax_list[i].set_ylim([-0.01,2.5])
    # vel_error
    if 3==plot_param_list[i][1]:
        ax_list[i].set_ylim([-0.01,1.0])

    if i==0:
        ax_list[i].legend(fontsize=13)
    if plot_param_list[i][0]=="vehicle_type":
        ax_list[i].text(0.5, 0.8, "0: nominal vehicle\n1: heavy-weight bus\n2: light-weight bus\n3: small vehicle\n4: small robot", va='center', ha='center', fontsize=15, transform=ax_list[i].transAxes)
plt.savefig("auto_test_result.png")
plt.show()